# 读取数据

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
train_data=pd.read_csv('../input/train.csv',sep=',')

In [ ]:
train_data.shape

In [ ]:
    train_data.columns

In [ ]:
train_data.isnull().sum()#检查是否有空值，根据结果查看是没有的

In [ ]:
train_data['Cover_Type'].value_counts() #查看预测值的分布

In [ ]:
test_data=pd.read_csv('../input/test.csv',sep=',')

In [ ]:
test_data.shape

In [ ]:
test_data.head()

# 处理离群值

In [ ]:
#3 
corr = train_data.corr()
f, ax = plt.subplots(figsize=(10, 8))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5)

#4
train_data.boxplot(rot = 90, figsize=(12,10))
plt.scatter(x = train_data.loc[:,'Hillshade_3pm'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()
train_data.drop(train_data[(train_data.loc[:,'Hillshade_3pm']< 10)&(train_data.iloc[:,-1]==1)].index, inplace=True)

plt.scatter(x = train_data.loc[:,'Hillshade_Noon'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()
train_data.drop(train_data[(train_data.loc[:,'Hillshade_Noon']< 150)&(train_data.iloc[:,-1]==2)].index, inplace=True)
train_data.drop(train_data[(train_data.loc[:,'Hillshade_Noon']< 120)&(train_data.iloc[:,-1]==1)].index, inplace=True)
train_data.drop(train_data[(train_data.loc[:,'Hillshade_Noon']< 100)&(train_data.iloc[:,-1]==7)].index, inplace=True)

plt.scatter(x = train_data.loc[:,'Slope'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()

train_data.drop(train_data[(train_data.loc[:,'Slope']> 45)&(train_data.iloc[:,-1]==6)].index, inplace=True)
train_data.drop(train_data[(train_data.loc[:,'Slope']> 40)&(train_data.iloc[:,-1]==5)].index, inplace=True)

plt.scatter(x = train_data.loc[:,'Vertical_Distance_To_Hydrology'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()

train_data.drop(train_data[(train_data.loc[:,'Vertical_Distance_To_Hydrology']> 500)].index, inplace=True)
train_data.drop(train_data[(train_data.loc[:,'Vertical_Distance_To_Hydrology']<-150)].index, inplace=True)

plt.scatter(x = train_data.loc[:,'Horizontal_Distance_To_Hydrology'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()

plt.scatter(x = train_data.loc[:,'Horizontal_Distance_To_Roadways'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()

train_data.drop(train_data[(train_data.loc[:,'Horizontal_Distance_To_Roadways']> 4000)&(train_data.iloc[:,-1]==5)].index, inplace=True)

plt.scatter(x = train_data.loc[:,'Horizontal_Distance_To_Fire_Points'], y = train_data.iloc[:,-1])
plt.grid(True)
plt.show()

train_data.drop(train_data[(train_data.loc[:,'Horizontal_Distance_To_Fire_Points']> 3600)&(train_data.iloc[:,-1]==5)].index, inplace=True)


# 分割数据集

In [ ]:
train_data.drop(['Soil_Type15' , "Soil_Type7"], inplace = True, axis = 1 )
test_data.drop(['Soil_Type15' , "Soil_Type7"], inplace = True, axis = 1 )
train_x,val_x,train_y,val_y=train_test_split(train_data.iloc[:,1:-1].values,train_data.iloc[:,-1].values,test_size=0.2)

In [ ]:
train_y=train_y.reshape(-1,1)
val_y=val_y.reshape(-1,1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier(max_depth=20,random_state=0,n_estimators=500)
model.fit(train_x,train_y)

In [ ]:
train_y_pred=model.predict(train_x)

In [ ]:
print(classification_report(train_y,train_y_pred))

In [ ]:
val_y_pred=model.predict(val_x)

In [ ]:
print(classification_report(val_y,val_y_pred))

In [ ]:
test_y_pred=model.predict(test_data.iloc[:,1:].values)

In [ ]:
test_data['Cover_Type']=test_y_pred

In [ ]:
test_submission=test_data[["Id",'Cover_Type']]

In [ ]:
test_submission.to_csv('../working/test_submission.csv',index=False)